In [1]:
import requests
import re
import os
from bs4 import BeautifulSoup as bsp, Tag
import pandas as pd

# Get urls

In [37]:
html = 'https://glt.goarch.org/'
page_response = requests.get(html)
page_html = page_response.text
soup = bsp(page_html)
# get URL list
url_list = [item.get('href') for item in soup.find_all('a')]
# URL list cleanup
urls_list_clean = [x for x in url_list if x is not None]
urls_list_clean = [x for x in urls_list_clean if 'Pen' in x]
urls_list_clean = [(html + x) for x in urls_list_clean] # needs revision

In [45]:
# !mkdir 'data/html'

Save htmls locally:

In [63]:
for index, url in enumerate(urls_list_clean):
    page_response = requests.get(url)
    page_response.encoding = page_response.apparent_encoding # solving encoding issue
    page_html = page_response.text
    with open(f'data/html/{index}.html', 'w', encoding='utf-8') as localfile:
        localfile.write(page_html)

# Cleanup htmls:

Deleting the faulty unclosed `<font size='3'>` tag 

In [126]:
## testing with a single file: successful

with open('data/html/0.html', 'r', encoding='utf-8') as local_html:
    html_text = local_html.read()
    soup = bsp(html_text)
    soup.font.unwrap()

with open('data/html/0 copy.html', 'w', encoding='utf-8') as local_html:
    local_html.write(str(soup))

In [136]:
## batch processing: succesfull

directory = 'data/html/'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and filename != '.DS_Store':
        try:
            with open(f, 'r', encoding='utf-8') as local_html:
                html_text = local_html.read()
                soup = bsp(html_text)
                soup.font.unwrap()
        except:
            raise ValueError(f'File {f} could not be processed')
        try:
            with open(f, 'w', encoding='utf-8') as local_html:
                local_html.write(str(soup))
        except:
            raise ValueError(f'File {f} could not be written')

# Parse htmls:

In [94]:
# single file

with open('data/html/0.html') as local_html:
    html_text = local_html.read()
    soup = bsp(html_text)
    
print(soup.prettify())

<!DOCTYPE HTML>
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="CONTENT-TYPE"/>
  <title>
   ΠΑΣΧΑ
  </title>
  <base target="_blank"/>
 </head>
 <body dir="LTR" lang="el-gr">
  <p align="center">
   <a href="http://www.goarch.org/" target="_blank">
    <img align="left" america="" archdiocese="" border="0" height="100" name="Greek" of="" orthodox="" src="../../images/ArchdioceseSeal.jpg" width="100"/>
   </a>
  </p>
  <br/>
  <br/>
  <br/>
  <br/>
  <br/>
  <br/>
  <font color="#ff0000">
   ΕΩΘΙΝΟΝ Ζ' Ἦχος βαρὺς
  </font>
  <br/>
  Ἰδοὺ σκοτία καὶ πρωΐ, καὶ τί πρὸς τὸ μνημεῖον Μαρία ἕστηκας, πολὺ σκότος ἔχουσα ταῑς φρεσίν; ὑφ' οὗ ποῦ τέθειται ζητεῖς ὁ Ἰησοῦς. Ἀλλ' ὅρα τοὺς συντρέχοντας Μαθητάς, πῶς τοῖς ὀθονίοις καὶ τῷ σουδαρίῳ, τὴν Ἀνάστασιν ἐτεκμήραντο, καὶ ἀνεμνήσθησαν τῆς περὶ τούτου Γραφῆς. Μεθ' ὧν, καὶ δι' ὧν καὶ ἡμεῖς, πιστεύσαντες, ἀνυμνοῦμέν σε τὸν ζωοδότην Χριστόν.
  <br/>
  <br/>
  <font color="#ff0000">
   Ἦχος πλ. α'
  </font>
  <br/>
  <font color="

In [230]:
# parsing function
data = []

def getdata(soup):
    for child in soup.body.children:
        if type(child) == Tag:
            if child.name == 'font':
                if 'color' in child.attrs:
                    output = child.text.strip(' \t\n\r')
                    df_row = {'red_text': output, 'color': 'red'}
                    data.append(df_row) #DRY optimization required
                else:
                    output = child.text.strip(' \t\n\r')
                    df_row = {'black_text': output, 'color': 'black'}
                    data.append(df_row)
        else:
            if len(child.strip()) > 0:
                output = child.strip(' \t\n\r')
                df_row = {'black_text': output, 'color': 'black'}
                data.append(df_row)

# Important revision

`os.listdir()` -> `sorted(os.listdir())`

In [232]:
directory = 'data/html/'
for filename in sorted(os.listdir(directory)):
    f = os.path.join(directory, filename)
    print(filename)

.DS_Store
00.html
02.html
04.html
05.html
06.html
07.html
08.html
09.html
10.html
11.html
12.html
13.html
14.html
15.html
16.html
17.html
18.html
19.html
20.html
21.html
22.html
23.html
24.html
25.html
26.html
27.html
28.html
29.html
30.html
31.html
32.html
33.html
34.html
35.html
36.html
37.html
38.html
39.html
40.html
41.html
42.html
43.html
44.html
45.html
46.html
47.html
48.html
49.html
50.html
51.html
52.html
53.html
54.html
55.html
56.html
57.html
58.html
special


In [238]:
#batch parsing, succesful

directory = 'data/html/'

for filename in sorted(os.listdir(directory)):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and filename != '.DS_Store':
        with open(f, 'r', encoding='utf-8') as local_html:
            html_text = local_html.read()
            soup = bsp(html_text)
            getdata(soup)

In [239]:
## exporting original data to df
df = pd.DataFrame(data, columns=['red_text', 'black_text', 'color'])
df = df.fillna('') #replace NaN fields with empty string

In [147]:
df

,red_text,black_text,color,temp,service
0,ΤΗΣ ΔΙΑΚΑΙΝΗΣΙΜΟΥ,,red,1,
1,Τῌ ΑΓΙᾼ ΚΑΙ ΜΕΓΑΛῌ ΚΥΡΙΑΚῌ ΤΟΥ ΠΑΣΧΑ,,red,1_1,
2,ΕΩΘΙΝΟΝ Ζ' Ἦχος βαρὺς,,red,,temp
3,,"Ἰδοὺ σκοτία καὶ πρωΐ, καὶ τί πρὸς τὸ μνημεῖον ...",black,,
4,Ἦχος πλ. α',,red,,
...,...,...,...,...,...
15561,,Καὶ νῦν...,black,,
15562,Θεοτοκίον,,red,,
15563,,"Χαῖρε τοῦ Θεοῦ, εὐρύχωρον χωρίον. Χαῖρε κιβωτέ...",black,,
15564,Κοινωνικὸν,,red,,


# CSV cleanup

In [245]:
# clean ΤΟ ΑΚΟΥΤΕ

# cleanup strings == το ακουτε
df = df[df.red_text != 'ΤΟ ΑΚΟΥΤΕ']
df = df.reset_index(drop=True)

In [248]:
## cleanup the strings containing το ακουτε

df[df['red_text'].str.contains('ΤΟ ΑΚΟΥΤΕ')] # check for strings to replace

# df = df.replace(' ΤΟ ΑΚΟΥΤΕ', '', regex=True)
# df = df.replace('ΤΟ ΑΚΟΥΤΕ', '', regex=True) # exception

,red_text,black_text,color


In [249]:
## stripping linebreak

df.red_text.replace('\n', ' ', regex=True, inplace=True)

In [250]:
#check for strings
df[df['red_text'].str.contains('\n', regex=True)==True]

,red_text,black_text,color


In [253]:
# s p a c e d characters cleanup
reg1 = '((\w\s){2,})'
# df[df['red_text'].str.match(reg1)==True]
[i[0][0] for i in df['red_text'].str.findall(reg1, flags=re.M) if len(i)>0]

['α ἡ ',
 'ς ὁ ',
 'ω ὁ ',
 'η ἡ ',
 'α ὁ ',
 'ν ἡ ',
 'ἶ ὁ ',
 'ἶ ὁ ',
 'υ ἡ ',
 'ι ὁ ',
 'ὲ ἡ ',
 'ς ὁ ',
 'ν ὁ ',
 'ι ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'η ἡ ',
 'υ ἡ ',
 'ς ὁ ',
 'ς ὁ ',
 'ὗ ἡ ',
 'ῦ ἡ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ν ὁ ',
 'ὗ ἡ ',
 'ς ὁ ',
 'η ἡ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'η ἡ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'η ἡ ',
 'ν ἡ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ὗ ἡ ',
 'η ἡ ',
 'ν ἡ ',
 'η ἡ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'η ἡ ',
 'ν ἡ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ν ὁ ',
 'ὗ ἡ ',
 'η ἡ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ὗ ἡ ',
 'ὗ ἡ ',
 'ς ὁ ',
 'ς ὁ ',
 'ὗ ἡ ',
 'ς ὁ ',
 'ι ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ς ὁ ',
 'ὗ ἡ ',
 'ῳ ὁ ']

In [252]:
df = df.replace(regex=r'Λ ι τ ὴ ν', value='Λιτὴν')
df = df.replace(regex=r'Α ἲ ν ο υ ς', value='Αἲνους')
df = df.replace(regex=r'Σ Υ Ν Α Ξ Α Ρ Ι Ο Ν', value='ΣΥΝΑΞΑΡΙΟΝ')
df = df.replace(regex=r'Σ τ ί χ ο ν', value='Στίχον')
df = df.replace(regex=r'Τ ρ ι σ α γ ί ο υ', value='Τρισαγίου')

Might've used `.strip()`

In [256]:
## cleanup lowercaseUPPERCASE merger
# checker:

reg2 = r'([α-ω]|ῼ|ῌ)([Α-Ω]|ῼ|ῌ)'
[i for i in df['red_text'].str.findall(reg2, flags=re.M) if len(i)>0]

[]

In [255]:
## cleaner

# mask = df['red_text'].str.extract(reg2)
# df[mask].notnull()
# df.red_text.str.findall(reg2)
df.red_text.replace({r'([α-ω]|ῼ|ῌ)([Α-Ω]|ῼ|ῌ)' : r'\1 \2'}, regex=True, inplace=True)

### NB!
The regex in the snippet below didn't work as expected (i.e. did not split groups with a whitespace), since it was missing an `r` flag, and therefore the interpertator treated `\1 \2` as ASCII characters

```
reg_replace = '\1 \2'
df = df.replace(regex={reg2: reg_replace})
```